<a href="https://colab.research.google.com/github/marquesguil/geocode/blob/master/Geocode_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install unidecode
!pip install geopy

     |████████████████████████████████| 245kB 4.8MB/s 


In [4]:
import re
import pandas as pd
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
from unicodedata import normalize

In [5]:
df = pd.read_csv('/content/drive/My Drive/Zeta_Project/amostragem.csv')
df.columns

Index(['Razao Social', 'Nome Fantasia', 'Logradouro', 'numero', 'Bairro',
       'Municipio', 'cep', 'UF'],
      dtype='object')

In [6]:
colunas = ['Logradouro','Latitude','Longitude']
dfNew = pd.DataFrame(columns=colunas)

In [7]:
dfNew.columns

Index(['Logradouro', 'Latitude', 'Longitude'], dtype='object')

In [8]:
def garbageCleanerT(texto):
    texto = texto.lower()
    texto = re.sub('[0-9]', '', texto)
    texto = re.sub('[.]', '', texto)
    texto = re.sub(r"http\S+", '', texto)
    texto = re.sub(u'[!@#$%¨<&*(){}|/\_,.;:>?"+-]','',texto)
    texto = re.sub(r"(\s)+",r' ',texto) # Remove espaços duplos
    return(normalize('NFKD', texto).encode('ASCII', 'ignore').decode('ASCII'))

In [9]:
def garbageCleanerN(numero):
    numero = re.sub('[^0-9]', '', numero)
    numero = re.sub('[.]', '', numero)
    numero = re.sub(r"http\S+", '', numero)
    numero = re.sub(u'[!@#$%¨<&*(){}|/\_,.;:>?"+-]','',numero)
    numero = re.sub(r"(\s)+",r' ',numero) # Remove espaços duplos
    return(numero)

In [10]:
i = 0
for ix, ln in df.iterrows():
  try:
    dirtyText = df.Logradouro[i]
    texto = garbageCleanerT(dirtyText)
    df.Logradouro[i] = texto
    i += 1
  except:
    i += 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [11]:
i = 0
for ix, ln in df.iterrows():
  try:
    dirtyNumber = df.numero[i]
    number = garbageCleanerN(dirtyNumber)
    df.numero[i] = number
    i += 1
  except:
    i += 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [12]:
i = 0
for ix, ln in df.iterrows():
  try:
    dirtyText = df.Bairro[i]
    texto = garbageCleanerT(dirtyText)
    df.Bairro[i] = texto
    i += 1
  except:
    i += 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [13]:
i = 0
for ix, ln in df.iterrows():
  try:
    dirtyText = df.Municipio[i]
    texto = garbageCleanerT(dirtyText)
    df.Municipio[i] = texto
    i += 1
  except:
    i += 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [14]:
dfNew['Logradouro'] = df['Logradouro']+', '+df['Municipio']+', '+df['UF']

In [15]:
dfNew

,Logradouro,Latitude,Longitude
0,"antonio ovidio rodrigues, jundiai, SP",NaN,NaN
1,"anhanguera, ribeirao preto, SP",NaN,NaN
2,"dos autonomistas, osasco, SP",NaN,NaN
3,"estevao mendonca, sao paulo, SP",NaN,NaN
4,"menina moca, olimpia, SP",NaN,NaN
...,...,...,...
9997,"jornalista roberto marinho, sao paulo, SP",NaN,NaN
9998,"nicola demarchi, sao bernardo do campo, SP",NaN,NaN
9999,"presidente juscelino kubitschek, sao paulo, SP",NaN,NaN
10000,"noboru kihara, pompeia, SP",NaN,NaN


In [16]:
locator = Nominatim(user_agent = 'geocode_zeta')

In [17]:
geocode = RateLimiter(locator.geocode, min_delay_seconds=0.1)

In [ ]:
i = 0 
for ix, ln in dfNew.iterrows():
    try:
        location = locator.geocode (dfNew.Logradouro[i])    
        dfNew.Latitude[i] = (location.latitude)
        dfNew.Longitude[i] = (location.longitude)
        i += 1
    except:
        i += 1

In [ ]:
dfNew.to_csv('/content/drive/My Drive/Zeta_Project/out_amostragem.csv', encoding='utf-8', index=False)

In [ ]:
dfPlot = dfNew.dropna(subset=['Latitude'], inplace=True)